In [1]:
import sys
!{sys.executable} -m pip install sodapy

You are using pip version 9.0.3, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
from sodapy import Socrata

In [3]:
client = Socrata('data.cityofnewyork.us', None)
columns = ['borough', 'complaint_type', 'incident_zip']

In [4]:
def get_records(limit, offset):
    records = client.get(
        'fhrw-4uyv',
        select=', '.join(columns),
        where='''\
            created_date >= "2017-01-01T00:00:00"
            and created_date < "2018-01-01T00:00:00"
            ''',
        order='created_date',
        limit=limit,
        offset=offset)
    
    return records

In [5]:
df = pd.DataFrame(columns=columns)
offset = 0; limit = 100000
prev_length = -1

In [6]:
# page through the data
while len(df) != prev_length:
    records = get_records(limit, offset)

    prev_length = len(df)
    df = pd.concat([df, pd.DataFrame.from_records(records)], ignore_index=True)
    
    offset += limit

In [7]:
# Create a new DataFrame ttc (top ten complaints) using boolean indexing to select 10 most common complaints
topTenComplaints = df['complaint_type'].value_counts().nlargest(10).index
ttc = df[df['complaint_type'].isin(topTenComplaints)]
ttc.head()

,borough,complaint_type,incident_zip
35,BRONX,HEAT/HOT WATER,10467
36,QUEENS,Blocked Driveway,11368
37,BROOKLYN,Noise - Residential,11209
38,MANHATTAN,Noise - Residential,10040
39,BRONX,HEAT/HOT WATER,10470


In [8]:
# The following contingency table shows the number of complaints by type for each borough, 
# for the top ten most common types of complaints
pd.crosstab(index=ttc['complaint_type'], columns=ttc['borough'], margins=True)

borough,BRONX,BROOKLYN,MANHATTAN,QUEENS,STATEN ISLAND,Unspecified,All
complaint_type,,,,,,,
Blocked Driveway,24574,49302,3428,54290,3465,1038,136097
HEAT/HOT WATER,68718,66984,46529,29217,2073,0,213521
Illegal Parking,16122,55380,19687,46065,7574,1294,146122
Noise,3134,15421,29002,10685,1926,3,60171
Noise - Residential,57663,67629,51026,46396,6744,694,230152
Noise - Street/Sidewalk,14025,21313,29147,7530,853,217,73085
Street Condition,11761,25432,14840,30629,10560,43,93265
Street Light Condition,18411,22458,11077,24258,6604,1388,84196
UNSANITARY CONDITION,24561,26659,14635,11474,1953,0,79282


In [9]:
# Use Splitwise's census data for population by zip
# https://blog.splitwise.com/2013/09/18/the-2010-us-census-population-by-zip-code-totally-free/
# Cast zip codes as strings for proper comparison
pz = pd.read_csv('population_by_zip_2010.csv', dtype={'Zip Code ZCTA': 'str'})
pz = pz.rename(columns={'Zip Code ZCTA': 'incident_zip', '2010 Census Population': 'population'})
pz.head()

,incident_zip,population
0,01001,16769
1,01002,29049
2,01003,10372
3,01005,5079
4,01007,14649


In [10]:
# Map populations onto zip codes relevant to NYC
ttc = ttc.merge(pz, how='left', on=['incident_zip'])
ttc.head()

,borough,complaint_type,incident_zip,population
0,BRONX,HEAT/HOT WATER,10467,97060.0
1,QUEENS,Blocked Driveway,11368,109931.0
2,BROOKLYN,Noise - Residential,11209,68853.0
3,MANHATTAN,Noise - Residential,10040,41905.0
4,BRONX,HEAT/HOT WATER,10470,15293.0


In [11]:
# Drop duplicate zip codes and then find the 10 most populous ones
topTenPopulationIndices = ttc.loc[ttc['incident_zip'].drop_duplicates().index]['population'].nlargest(10).index
topTenZips = ttc.loc[topTenPopulationIndices, 'incident_zip'].values
topTenZips

array(['11368', '11226', '11373', '11220', '11385', '10467', '10025',
       '11208', '11236', '11207'], dtype=object)

In [12]:
# Create new DataFrame ttcz (top ten complaints and zip codes) containing 
# 10 most common complaints occurring in the ten most populous zip codes
ttcz = ttc[ttc['incident_zip'].isin(topTenZips)]
ttcz.head()

,borough,complaint_type,incident_zip,population
0,BRONX,HEAT/HOT WATER,10467,97060.0
1,QUEENS,Blocked Driveway,11368,109931.0
15,QUEENS,Noise - Residential,11373,100820.0
21,QUEENS,Blocked Driveway,11368,109931.0
34,QUEENS,Noise - Residential,11373,100820.0


In [13]:
# The following contingency table shows the number of complaints by type for each zip code, 
# for the top ten most common types of complaints and then top ten most populous zip codes
pd.crosstab(index=ttcz['complaint_type'], columns=ttcz['incident_zip'], margins=True)

incident_zip,10025,10467,11207,11208,11220,11226,11236,11368,11373,11385,All
complaint_type,,,,,,,,,,,
Blocked Driveway,125,2068,2062,2756,1558,2203,3041,4384,2635,3042,23874
HEAT/HOT WATER,2397,6041,2461,2052,1634,7569,1145,1620,3408,1526,29853
Illegal Parking,736,986,1500,2150,2013,1076,1431,1251,1277,4135,16555
Noise,1398,285,199,188,442,440,108,158,270,507,3995
Noise - Residential,2085,5807,3061,2795,1522,4854,1929,2460,1842,2609,28964
Noise - Street/Sidewalk,1224,713,558,825,498,1831,201,684,304,607,7445
Street Condition,628,612,1142,816,690,491,1021,561,691,1232,7884
Street Light Condition,21,407,932,566,610,202,180,444,362,791,4515
UNSANITARY CONDITION,714,2192,1621,1341,719,3155,562,639,756,647,12346
